# Cargar librerias

In [6]:
# Montar drive y crear path para sys y para directorio de trabajo

from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# Define el path al directorio de trabajo actual
path = "/content/drive/My Drive/cod/analitica_marketing"


sys.path.append(path) ### para importar archivo de funciones propias a través de import

os.chdir(path) ### para que por defecto suba y descargue archivos partiendo de esa ruta

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl


####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

In [7]:
import a_funciones as funciones     # Importar las funciones
import importlib
importlib.reload(funciones)

<module 'a_funciones' from '/content/drive/My Drive/cod/analitica_marketing/a_funciones.py'>

In [1]:
####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

import logging
from tqdm import tqdm


In [5]:
# Conffiguración del logg
logging.basicConfig(
    filename='G:\Mi unidad\cod\analitica_marketing\salida\reco\script_log.log',
    level=logging.INFO,       #nivel de logg, informativo solo muestra informació básica de ejecución
    format='%(asctime)s - %(levelname)s - %(message)s'  ## formato log, hora, nievel(error, informativo, advertencia), mensaje personalizado cuando se llama logg
)

In [17]:
def preprocesar(conn=None, cur=None):

    ######## convertir datos crudos a bases filtradas por usuarios que tengan cierto número de calificaciones
    funciones.ejecutar_sql('G:\Mi unidad\cod\analitica_marketing\preprocesamiento.sql', cur)

    log_mes='Ejecución de SQL para filtrar peliculas y ratings completada.'
    logging.info(log_mes)
    print(log_mes)

    ##### llevar datos que cambian constantemente a python ######
    movies1=pd.read_sql("""select * from db_movies_final""", conn)
    ratings=pd.read_sql("""select * from db_movies_final""", conn)
    usuarios=pd.read_sqlpd.read_sql('select distinct (user_id) as user_id from db_movies_final',conn)

    #### transformación de datos crudos - Preprocesamiento ################

    # Se eliminan los registros duplicados, solo se requiere el catalogo de las peliculas
    movies2 = movies1.drop_duplicates(subset='movie_Id', keep='first').reset_index(drop=True)

    ##### escalar para que año esté en el mismo rango ###
    sc=MinMaxScaler()
    movies2[["movie_yearsc"]]=sc.fit_transform(movies2[['movie_year']])

    #eliminar columnas que no se requieren para el analisis
    movies3=movies2.drop(columns=['user_id','movie_Id','movie_rating','movie_year','rating_year','movie_title'])


    log_mes='Preprocesamiento de datos completado.'
    logging.info(log_mes)
    print(log_mes)

    return movies3,movies2, conn, cur


In [ ]:
##########################################################################
###############Función para entrenar modelo por cada usuario ##########
###############Basado en contenido todo lo visto por el usuario Knn#############################
def recomendar(user_id, conn=None, cur=None, books_dum2=None, books=None):

    ratings=pd.read_sql('select *from db_movies_final where user_id=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movie_id'].to_numpy()
    movies3[['movie_id','movie_title']]=movies2[['movie_id','movie_title']]
    movies_r=movies3[movies3['movie_id'].isin(l_books_r)]
    movies_r=movies_r.drop(columns=['movie_id','movie_title'])
    books_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    centroide=books_r.groupby("indice").mean()

    log_mes=f'Generando recomendaciones para el usuario {user_id}.'
    logging.info(log_mes)
    print(log_mes)




    books_nr=books_dum2[~books_dum2['isbn'].isin(l_books_r)]
    books_nr=books_nr.drop(columns=['isbn','book_title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(books_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0]
    recomend_b=books.loc[ids][['book_title','isbn']]

    log_mes=f'Recomendaciones para el usuario {user_id} finalizadas'
    logging.info(log_mes)
    print(log_mes)

    return recomend_b